In [ ]:
from itertools import product
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
# from torch.nn.functional import relu
# from torch.nn.functional import tanh
# from torch.nn.functional import sigmoid
from scipy.fftpack import dct
from scipy.signal import convolve2d
from skimage.measure import block_reduce
from sklearn import svm

## Reduce the data size by shrinking the image and allow symbolic dynamics to occur (based on doppler-effect).

### Load the data

In [2]:
train_data = datasets.MNIST(
    root='data',
    train=True,
    transform=ToTensor(),
    download=True,
)
test_data = datasets.MNIST(
    root='data',
    train=False,
    transform=ToTensor(),
)
data_len = {'train': len(train_data), 'test': len(test_data)}

loaders = {
    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),

    'test': DataLoader(test_data,
                       batch_size=100,
                       shuffle=True,
                       num_workers=1),
}

### Symbolic dynamics through iteration and non-linearity.
Through multiple iterations apply the doppler effect to the DCT of the images and apply convolution.  
Save the results in a dataframe for easy loading.

In [3]:
relu = np.vectorize(lambda x: np.maximum(0, x))
# sigmoid = np.vectorize(lambda x: 1 / (1 + np.exp(-x)))
tanh = np.vectorize(np.tanh)

In [14]:
def apply_dynamics(sample, iters, f_act, vel, v_o, v_m, kernel, conv=False, pool=False):
    working_sample = np.copy(sample)

    for i in range(iters):
        v_s = vel(i, iters)
        # apply doppler effect to sample
        working_sample = ((v_m + v_o) / (v_m + v_s)) * working_sample  # doppler-effect
        working_sample = f_act(working_sample)  # activation function

        if conv:
            working_sample = convolve2d(working_sample, kernel, mode='valid')  # convolution

        # if pool and i % 4 == 1:
        #     working_sample = block_reduce(working_sample, (2, 2), np.mean, cval=0.5)

    return working_sample

###### Visualization
Changing `iters` determines the size of the output (if conv and/or pooling are on).

In [18]:
samples, labels = next(iter(loaders['train']))
sample = samples[0][0]

# vel_s is negative if moving towards observer
def v1(x, n):
    return -(x + 1) / (n / 3)
def v2(x, n):
    return x
def v3(x, n):
    return 1.5

v_o = 0  # positive if moving towards source
v_m = 5.022  # small tail to avoid division errors.
kernel = np.array([[2.5, 2.5],
                   [2.5, 2.5]])

In [21]:
iters = 24

freq_sample = dct(dct(sample.numpy().T, norm='ortho').T, norm='ortho')  # decompose sample
print(f"freq_sample {freq_sample.shape}\n{np.round(freq_sample, 2)}")
result = apply_dynamics(freq_sample, iters, tanh, v2, v_o, v_m, kernel, conv=True)
print(f"result {result.shape}\n{np.round(result, 2)}")

freq_sample (28, 28)
[[ 3.19 -0.45 -2.17  0.99 -1.19 -0.85  1.31  0.41  0.07 -0.1  -0.15 -0.05
  -0.28  0.17  0.16 -0.18 -0.06  0.09  0.13 -0.01 -0.   -0.01 -0.12  0.02
   0.07  0.01 -0.   -0.09]
 [ 0.02  0.63  0.5  -1.25 -0.8   0.92 -0.12 -0.45  0.84  0.2  -0.5   0.01
   0.12 -0.14 -0.19  0.12  0.14 -0.1   0.03  0.11 -0.04 -0.06 -0.01  0.01
  -0.01  0.    0.01 -0.01]
 [-2.44  0.35  1.25 -0.34  1.79 -0.44 -1.43  0.7  -0.56 -0.31  1.01  0.1
  -0.39 -0.12  0.11 -0.01 -0.04  0.16 -0.09 -0.13  0.03  0.1   0.09 -0.1
  -0.06  0.01  0.    0.11]
 [-1.58 -0.71  0.86  1.23  0.6  -0.38  0.29 -0.57 -1.07  0.74  0.09 -0.42
   0.67  0.09 -0.3   0.1   0.05 -0.12 -0.16  0.07  0.06 -0.04  0.07  0.03
  -0.04 -0.04  0.01  0.06]
 [ 0.58 -0.46 -0.39  0.29 -0.31  0.76  0.22 -0.52  0.6  -0.57 -0.92  0.68
   0.24 -0.14  0.44  0.02 -0.43 -0.05  0.18 -0.08 -0.07  0.07 -0.04  0.03
   0.1   0.   -0.06 -0.09]
 [ 1.88  0.06 -1.45 -0.21 -0.23  0.03  0.27  0.62  0.12 -0.71  0.32 -0.12
  -0.58  0.66  0.09 -0.38  0.28 

###### Convert all data
Changing `iters` determines the size of the output.

In [25]:
iters = 18
f_act = tanh

df_results = pd.DataFrame(columns=['data', 'label', 'train'])
for dset in ["train", "test"]:
    batch_size = 100
    batches = data_len[dset] / batch_size
    print(f"Begin reducing {dset}")
    for b, (images, labels) in enumerate(loaders[dset]):
        if b == batches:
            break
        for i in range(batch_size):
            freq_sample = dct(dct(images[i].numpy()[0].T, norm='ortho').T, norm='ortho')  # decompose sample
            result = apply_dynamics(freq_sample, iters, f_act, v2, v_o, v_m, kernel, conv=True)
            result_str = np.array2string(result, separator=',')
            df_results = pd.concat([df_results, pd.DataFrame([[result_str, labels[i].numpy(), dset == "train"]], columns=df_results.columns)], ignore_index=True)

        if (b + 1) % 30 == 0:
            print('Reduced [{}/{}] {} batches for {} iterations'
                  .format(b + 1, batches, dset, iters))

print(df_results.shape)
df_results.to_csv(f'freq_RedData/freq_RedData_{iters}i_conv_tanh.csv', index=False)

Begin reducing train
Reduced [30/600.0] train batches for 18 iterations
Reduced [60/600.0] train batches for 18 iterations
Reduced [90/600.0] train batches for 18 iterations
Reduced [120/600.0] train batches for 18 iterations
Reduced [150/600.0] train batches for 18 iterations
Reduced [180/600.0] train batches for 18 iterations
Reduced [210/600.0] train batches for 18 iterations
Reduced [240/600.0] train batches for 18 iterations
Reduced [270/600.0] train batches for 18 iterations
Reduced [300/600.0] train batches for 18 iterations
Reduced [330/600.0] train batches for 18 iterations
Reduced [360/600.0] train batches for 18 iterations
Reduced [390/600.0] train batches for 18 iterations
Reduced [420/600.0] train batches for 18 iterations
Reduced [450/600.0] train batches for 18 iterations
Reduced [480/600.0] train batches for 18 iterations
Reduced [510/600.0] train batches for 18 iterations
Reduced [540/600.0] train batches for 18 iterations
Reduced [570/600.0] train batches for 18 itera